## GLIM embedding HMLN

In [ ]:
import sys
sys.path.append('../')
import glim
import numpy as np
import os
import glim.utils as utils

In [ ]:
relationship_file = 'relationship_table.txt'
embedding_save_file = '../results/multilayer_feature.npy'

In [ ]:
####################################################
#            MultiLayer Dataset                    #
####################################################
def load_multilayer(filepath='../data/'):
    X, A, Y = [], None, []
    n_node = 0

    # Acquire Edges
    edge_list = []
    node_list = []
    node_type = {}
    adj_path = filepath + relationship_file
    with open(adj_path, 'rt', encoding='utf-8') as f:
        next(f)
        for line in f.readlines():
            node1, node2, type1, type2, _ = line.strip().split('\t')
            edge_list.append((node1, node2))
            node_list.extend([node1, node2])
            if len(type1) != 1:
                type1 = 'c'
            if len(type2) != 1:
                type2 = 'c'
            node_type[node1] = type1
            node_type[node2] = type2
                
    node_map = {item:i for i, item in enumerate(sorted(list(set(node_list))))}
    n_node = len(node_map)
    A = np.zeros((n_node, n_node))
    for node1, node2 in edge_list:
        A[node_map[node1], node_map[node2]] = 1
        A[node_map[node2], node_map[node1]] = 1
    A = np.float32(A)
    
    ####################################################
    #            Acquire Labels                        #
    ####################################################
    Y = [node_type[item] for item in sorted(list(set(node_list)))]
    label_map = {item: i for i, item in enumerate(sorted(set(Y)))}
    Y = np.array([label_map[item] for item in Y])
    
    
    ####################################################
    #            Acquire Features                      #
    ####################################################
    feature_path = filepath + 'relationship_n2v.npy'
    if os.path.exists(feature_path):
        X = np.load(feature_path)
    else:
        X = np.float32(utils.N2V(A, 512, 4, 1))
        np.save(feature_path, X)
    
    return X, A, Y

loading HMLN graph network

In [ ]:
features, adj, labels = load_multilayer()
adj = adj + np.eye(adj.shape[0]) # self-loop is needed

GLIM embedding, outputs(npy) is saved to `embedding_save_file`

In [ ]:
glim.train.fit_transform(features, adj, embedding_save_file, device='cuda')

Save node's name and index

In [ ]:
filepath='../data/'
# Acquire Edges
edge_list = []
node_list = []
node_type = {}
adj_path = filepath + relationship_file
with open(adj_path, 'rt', encoding='utf-8') as f:
    next(f)
    for line in f.readlines():
        node1, node2, type1, type2,_ = line.strip().split('\t')
        edge_list.append((node1, node2))
        node_list.extend([node1, node2])
        if len(type1) != 1:
            type1 = 'c'
        if len(type2) != 1:
            type2 = 'c'
        node_type[node1] = type1
        node_type[node2] = type2

node_map = {item:i for i, item in enumerate(sorted(list(set(node_list))))}
n_node = len(node_map)

In [ ]:
import json
with open('../results/multilayer_node_map.json', 'wt') as f:
    json.dump(node_map, f)